File này: Với các điểm 3D được chuyển đổi trong ROI (output của file project_to_3d), dựa vào output này,   
ta nhìn vào ảnh point cloud để giữ lại các điểm trong ROI.

Hàm phía dưới là để xử lý với 1 ảnh

In [ ]:
# Ý tưởng:
# 1 Load scene point cloud .ply
# 2 load ROI point cloud (Xyz)
# 3 với mỗi điểm ROI -> tìm điểm gần nhất trong scene (KDTree Search)
# 4 Giữ các điểm trong scene thuộc ROI

# filter_ply.py

import numpy as np
import open3d as o3d

class PointCloudFilter:
    def __init__(self, distance_threshold=0.01):
        """
        @param distance_threshold: Ngưỡng để xác định điểm nào thuộc ROI (m)
        """
        self.distance_threshold = distance_threshold

    def filter_scene_with_roi(self, scene_ply_path, roi_pcd):
        """
        @param scene_ply_path: đường dẫn file PLY đầy đủ
        @param roi_pcd: Open3D ROI PointCloud
        @return: filtered_point_cloud (Open3D)
        """
        scene = o3d.io.read_point_cloud(scene_ply_path)

        # KDTree cho scene (tăng tốc độ tìm kiếm lân cận)
        kdtree = o3d.geometry.KDTreeFlann(scene)

        scene_points = np.asarray(scene.points)
        keep_mask = np.zeros(len(scene_points), dtype=bool)

        roi_points = np.asarray(roi_pcd.points)

        for roi_pt in roi_points:
            _, idx, dist = kdtree.search_knn_vector_3d(roi_pt, 1)
            if len(idx) > 0 and dist[0] < self.distance_threshold:
                keep_mask[idx[0]] = True

        filtered_scene = scene.select_by_index(np.where(keep_mask)[0])
        return filtered_scene

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Test thử với 1 ảnh

In [ ]:
# scene_path = "Public data/Public data train/ply/0002.ply"
# roi_path = "output/filtered_roi_0002.ply"

# roi = o3d.io.read_point_cloud(roi_path)
# filterer = PointCloudFilter(distance_threshold=0.01)

# filtered = filterer.filter_scene_with_roi(scene_path, roi)
# o3d.io.write_point_cloud("output/filtered_scene_0002.ply", filtered)

# print("✅ Filtered PLY saved: output/filtered_scene_0002.ply")

[Open3D WARNING] Read PLY failed: unable to open file: output\filtered_roi_0002.ply
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
✅ Filtered PLY saved: output/filtered_scene_0002.ply


Xử lý trên cả tập data

In [3]:
import os

In [4]:
# 1. Cấu hình
scene_folder = "Public data/Public data train/ply"       # thư mục chứa các file scene PLY
roi_file = "data/roi_pointclouds"          # file ROI PCD
output_folder = "data/filtered_ply"    # thư mục lưu kết quả
distance_threshold = 0.01

os.makedirs(output_folder, exist_ok=True)

# 2. Load ROI once
roi_pcd = o3d.io.read_point_cloud(roi_file)

# 3. Tạo object filter
pc_filter = PointCloudFilter(distance_threshold)

# 4. Lặp qua tất cả file PLY trong thư mục
for filename in os.listdir(scene_folder):
    if filename.endswith(".ply"):
        scene_path = os.path.join(scene_folder, filename)
        print(f"Processing {filename} ...")
        filtered_scene = pc_filter.filter_scene_with_roi(scene_path, roi_pcd)

        # 5. Lưu kết quả
        output_path = os.path.join(output_folder, filename)
        o3d.io.write_point_cloud(output_path, filtered_scene)
        print(f"Saved filtered point cloud to {output_path}")

[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for data\roi_pointclouds (format: auto).
Processing 0000.ply ...
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Saved filtered point cloud to data/filtered_ply\0000.ply
Processing 0001.ply ...
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Saved filtered point cloud to data/filtered_ply\0001.ply
Processing 0002.ply ...
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Saved filtered point cloud to data/filtered_ply\0002.ply
Processing 0003.ply ...
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Saved filtered point cloud to data/filtered_ply\0003.ply
Processing 0004.ply ...
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Saved filtered point cloud to data/filtered_ply\0004.ply
Processing 0005.ply ...
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Saved filtered point cloud to data/filtered_ply\0005.ply
Processing 0006.ply ...
[Open3D W